<a href="https://colab.research.google.com/github/pflor03/Pneumonia-Detection-using-Image-Classification/blob/main/Assignment4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Assignment 4**

Install Spark 

In [ ]:
# Update environment (if needed)
!sudo apt update

# Download and install Java
!sudo apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Install Apache Spark 3.0.3 with Hadoop 3.2
!wget -nc -q https://downloads.apache.org/spark/spark-3.0.3/spark-3.0.3-bin-hadoop3.2.tgz

# Unzip the folder
!tar xf  spark-3.0.3-bin-hadoop3.2.tgz

# Install findspark library that will locate Spark on the system
!pip install -q findspark

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad

In [ ]:
# Setting the environment variables, to enable running PySpark in Colab environment 
import os
import shutil
from itertools import islice
import requests

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.3-bin-hadoop3.2"

In [ ]:
# Locate Spark in the system

import findspark
findspark.init()

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
#from pyspark.sql import functions as F
from pyspark.sql.types import *

# Create spark_session
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

spark.version


'3.0.3'

In [ ]:
# pip uninstall pyspark

Found existing installation: pyspark 3.2.1
Uninstalling pyspark-3.2.1:
  Would remove:
    /usr/local/bin/beeline
    /usr/local/bin/beeline.cmd
    /usr/local/bin/docker-image-tool.sh
    /usr/local/bin/find-spark-home
    /usr/local/bin/find-spark-home.cmd
    /usr/local/bin/find_spark_home.py
    /usr/local/bin/load-spark-env.cmd
    /usr/local/bin/load-spark-env.sh
    /usr/local/bin/pyspark
    /usr/local/bin/pyspark.cmd
    /usr/local/bin/pyspark2.cmd
    /usr/local/bin/run-example
    /usr/local/bin/run-example.cmd
    /usr/local/bin/spark-class
    /usr/local/bin/spark-class.cmd
    /usr/local/bin/spark-class2.cmd
    /usr/local/bin/spark-shell
    /usr/local/bin/spark-shell.cmd
    /usr/local/bin/spark-shell2.cmd
    /usr/local/bin/spark-sql
    /usr/local/bin/spark-sql.cmd
    /usr/local/bin/spark-sql2.cmd
    /usr/local/bin/spark-submit
    /usr/local/bin/spark-submit.cmd
    /usr/local/bin/spark-submit2.cmd
    /usr/local/bin/sparkR
    /usr/local/bin/sparkR.cmd
    /usr/lo

In [ ]:
# !pip install pyspark


  Using cached pyspark-3.2.1-py2.py3-none-any.whl


Copy the file into your own directory in Google Drive
https://towardsdatascience.com/how-to-successfully-add-large-data-sets-to-google-drive-130beb320f1a



Method 1 :

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!unzip gdrive/My\ Drive/Municipal_Court_Caseload_Information.zip  

Archive:  gdrive/My Drive/Municipal_Court_Caseload_Information.zip
  inflating: Municipal_Court_Caseload_Information.csv  


Method 2:

In [31]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Mounted at /content/drive


In [29]:
def delete_folder(dir_name):
    try:
        shutil.rmtree(dir_name)
    except OSError as e:
        print("Error: %s : %s" % (dir_name, e.strerror))

In [37]:
def get_gcs_data (bucket_name, folder_name, file_name, path_gdrive):
    url = 'https://storage.googleapis.com/' + bucket_name + '/' + folder_name + '/' + file_name
    r = requests.get(url)
    open(path_gdrive + '/' + file_name , 'wb').write(r.content)

In [44]:
# Check the content of BDP folder in GDrive
!ls "/content/drive/My Drive/Colab Datasets/BDP/court/austin"

Municipal_Court_Caseload_Information.zip


In [45]:
bucket_name = 'msca-bdp-data-open'
folder_name = 'austin'
file_name = ['Municipal_Court_Caseload_Information.zip']
path_gdrive = '/content/drive/My Drive/Colab Datasets/BDP/court/austin'

os.makedirs(path_gdrive, exist_ok=True)

for file in file_name:
    get_gcs_data (bucket_name = bucket_name,
                 folder_name = folder_name,
                 file_name = file,
                 path_gdrive = path_gdrive)
    print('Downloaded: ' + file)

Downloaded: Municipal_Court_Caseload_Information.zip


Unzip the file (using Linux commands)


In [46]:
filename = '/content/drive/My Drive/Colab Datasets/BDP/court/austin/Municipal_Court_Caseload_Information.zip'
extract_dir = '/content/drive/My Drive/Colab Datasets/BDP/court/austin'
archive_format = 'zip'

shutil.unpack_archive(filename,extract_dir, archive_format)

In [58]:
!unzip drive/My\ Drive/Municipal_Court_Caseload_Information.zip  

Archive:  drive/My Drive/Municipal_Court_Caseload_Information.zip
replace Municipal_Court_Caseload_Information.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: Municipal_Court_Caseload_Information.csv  


In [49]:

readme_raw = sc.textFile("/content/drive/My Drive/Colab Datasets/BDP/court/austin/Municipal_Court_Caseload_Information.csv")
readme_raw.count()

8485777

In [50]:
# Check the content of BDP folder in GDrive
!ls "/content/drive/My Drive/Colab Datasets/BDP/court/austin"

Municipal_Court_Caseload_Information.csv
Municipal_Court_Caseload_Information.zip


In [47]:
! pwd

/content


Read the data into Spark RDD


In [ ]:
# Check the content of BDP folder in GDrive
!ls "/content"

gdrive					  spark-3.0.3-bin-hadoop3.2
Municipal_Court_Caseload_Information.csv  spark-3.0.3-bin-hadoop3.2.tgz
sample_data


In [ ]:
%%time

!ls -l "/content/Municipal_Court_Caseload_Information.csv"

-rw-r--r-- 1 root root 877080187 Apr 16  2015 /content/Municipal_Court_Caseload_Information.csv
CPU times: user 6.58 ms, sys: 4.26 ms, total: 10.8 ms
Wall time: 117 ms


In [54]:
%%time
readme = sc.textFile("/content/Municipal_Court_Caseload_Information.csv")
readme.count()

CPU times: user 56.8 ms, sys: 2.22 ms, total: 59 ms
Wall time: 14.3 s


Ensure your process the header record correctly


In [59]:
readme.take(1)

['Offense Case Type,Offense Date,Offense Time,Offense Charge Description,Offense Street Name,Offense Cross Street Check , Offense Cross Street,School Zone,Construction Zone,Case Closed']

In [55]:
readme.take(10)

['Offense Case Type,Offense Date,Offense Time,Offense Charge Description,Offense Street Name,Offense Cross Street Check , Offense Cross Street,School Zone,Construction Zone,Case Closed',
 'TR,04/29/2010 07:00:00 AM +0000,22.40.00,FAILED TO MAINTAIN FINANCIAL RESPONSIBILITY,8000 BLOCK RESEARCH,N, ,N,N,Y',
 'TR,04/29/2010 07:00:00 AM +0000,22.40.00,FAILURE TO SIGNAL INTENT TO CHANGE LANES,8000 BLOCK RESEARCH,N, ,N,N,Y',
 'TR,04/29/2010 07:00:00 AM +0000,20.00.00,SPEEDING-STATE HIGHWAYS,1000 BLOCK NORTH U S 183,N, ,N,N,Y',
 'TR,04/29/2010 07:00:00 AM +0000,20.00.00,NO SEAT BELT-DRIVER/PASSENGER,1000 BLOCK NORTH U S 183,N, ,N,N,Y',
 'TR,04/29/2010 07:00:00 AM +0000,23.45.00,SPEEDING - STATE HIGHWAYS - Less than 10% over limit,1000 BLOCK RESEARCH,N, ,N,N,Y',
 'PK,05/05/2010 07:00:00 AM +0000,15.28.00,PAY STATION RECEIPT NOT DISPLAYED,700 10TH ST W,N, ,N,N,Y',
 'PK,05/05/2010 07:00:00 AM +0000,15.23.00,EXPIRED PAY STATION RECEIPT,800 RIO GRANDE ST,N, ,N,N,Y',
 'PK,05/05/2010 07:00:00 AM +000

Calculate frequency of offenses by Offense Case Type




In [ ]:
casetype = readme.map(lambda s: s.split(",")[0]) 
casetype.first()
# airlines_dest = airlines_raw.map(lambda s: s.split(",")[17]) 

PythonRDD[43] at RDD at PythonRDD.scala:53

In [ ]:
a_orig = casetype.map(lambda x: (x, 1)).reduceByKey(lambda x,y:x+y).sortByKey(False)
a_orig.take(10)

[('TR', 4313221),
 ('PK', 3388981),
 ('Offense Case Type', 1),
 ('CM', 319078),
 ('CO', 240308),
 ('RL', 224188)]

Identify the most frequent offenses by Offense Charge Description (Show Offense Charge Description and offense frequency count in descending order)


In [ ]:
x1 = readme.map(lambda x: (x,1))
x1.take(10)

[('TR,04/29/2010 07:00:00 AM +0000,22.40.00,FAILED TO MAINTAIN FINANCIAL RESPONSIBILITY,8000 BLOCK RESEARCH,N, ,N,N,Y',
  1),
 ('TR,04/29/2010 07:00:00 AM +0000,22.40.00,FAILURE TO SIGNAL INTENT TO CHANGE LANES,8000 BLOCK RESEARCH,N, ,N,N,Y',
  1),
 ('TR,04/29/2010 07:00:00 AM +0000,20.00.00,SPEEDING-STATE HIGHWAYS,1000 BLOCK NORTH U S 183,N, ,N,N,Y',
  1),
 ('TR,04/29/2010 07:00:00 AM +0000,20.00.00,NO SEAT BELT-DRIVER/PASSENGER,1000 BLOCK NORTH U S 183,N, ,N,N,Y',
  1),
 ('TR,04/29/2010 07:00:00 AM +0000,23.45.00,SPEEDING - STATE HIGHWAYS - Less than 10% over limit,1000 BLOCK RESEARCH,N, ,N,N,Y',
  1),
 ('PK,05/05/2010 07:00:00 AM +0000,15.28.00,PAY STATION RECEIPT NOT DISPLAYED,700 10TH ST W,N, ,N,N,Y',
  1),
 ('PK,05/05/2010 07:00:00 AM +0000,15.23.00,EXPIRED PAY STATION RECEIPT,800 RIO GRANDE ST,N, ,N,N,Y',
  1),
 ('PK,05/05/2010 07:00:00 AM +0000,15.22.00,EXPIRED PAY STATION RECEIPT,800 RIO GRANDE ST,N, ,N,N,Y',
  1),
 ('PK,05/05/2010 07:00:00 AM +0000,15.18.00,EXPIRED PAY STATIO

In [60]:
charge = readme.map(lambda s: s.split(",")[3]) 
charge.take(10)


['Offense Charge Description',
 'FAILED TO MAINTAIN FINANCIAL RESPONSIBILITY',
 'FAILURE TO SIGNAL INTENT TO CHANGE LANES',
 'SPEEDING-STATE HIGHWAYS',
 'NO SEAT BELT-DRIVER/PASSENGER',
 'SPEEDING - STATE HIGHWAYS - Less than 10% over limit',
 'PAY STATION RECEIPT NOT DISPLAYED',
 'EXPIRED PAY STATION RECEIPT',
 'EXPIRED PAY STATION RECEIPT',
 'EXPIRED PAY STATION RECEIPT']

In [61]:
a1 = charge.map(lambda x: (x, 1))
a1.take(10)

[('Offense Charge Description', 1),
 ('FAILED TO MAINTAIN FINANCIAL RESPONSIBILITY', 1),
 ('FAILURE TO SIGNAL INTENT TO CHANGE LANES', 1),
 ('SPEEDING-STATE HIGHWAYS', 1),
 ('NO SEAT BELT-DRIVER/PASSENGER', 1),
 ('SPEEDING - STATE HIGHWAYS - Less than 10% over limit', 1),
 ('PAY STATION RECEIPT NOT DISPLAYED', 1),
 ('EXPIRED PAY STATION RECEIPT', 1),
 ('EXPIRED PAY STATION RECEIPT', 1),
 ('EXPIRED PAY STATION RECEIPT', 1)]

In [62]:
a2 = a1.reduceByKey(lambda x,y:x+y)
a2.take(10)

[('BICYCLE - RAN RED LIGHT', 2397),
 ('RAN RED LIGHT', 157783),
 ('CMV - UNSAFE CONDITION-396 3 A 1', 509),
 ('PARKING - SIDEWALK AREA', 9918),
 ('CAMPING IN A PUBLIC AREA', 12144),
 ('SEATBELT - PASSENGER', 3042),
 ('ALCOHOL - SELLING/POSSESSING IN PROHIBITED AREA', 3266),
 ('PARKING - DOUBLE PARKED', 2820),
 ('MOTOR VEHICLE INSPECTION VIOLATION', 43),
 ('CROSSING PROPERTY TO TURN RIGHT OR LEFT', 6866)]

In [63]:
a3 = a2.map(lambda x:(x[1],x[0]))
a3.take(10)

[(2397, 'BICYCLE - RAN RED LIGHT'),
 (157783, 'RAN RED LIGHT'),
 (509, 'CMV - UNSAFE CONDITION-396 3 A 1'),
 (9918, 'PARKING - SIDEWALK AREA'),
 (12144, 'CAMPING IN A PUBLIC AREA'),
 (3042, 'SEATBELT - PASSENGER'),
 (3266, 'ALCOHOL - SELLING/POSSESSING IN PROHIBITED AREA'),
 (2820, 'PARKING - DOUBLE PARKED'),
 (43, 'MOTOR VEHICLE INSPECTION VIOLATION'),
 (6866, 'CROSSING PROPERTY TO TURN RIGHT OR LEFT')]

In [64]:
a4 = a3.sortByKey(ascending=False)
a4.take(10)

[(892013, 'PAY STATION RECEIPT NOT DISPLAYED'),
 (732605, 'EXPIRED PAY STATION RECEIPT'),
 (486576, 'SPEEDING-STATE HIGHWAYS'),
 (372339, 'NO DRIVERS LICENSE'),
 (345162, 'SPEEDING - POSTED CITY STREET'),
 (337672, 'FAILED TO MAINTAIN FINANCIAL RESPONSIBILITY'),
 (310816, 'PARKING - EXPIRED METER'),
 (287570, 'SPEEDING - STATE HIGHWAY'),
 (260662, 'FAIL TO MAINTAIN FINANCIAL RESP'),
 (238168, 'TOW AWAY ZONE NO PARKING AREA')]